# Time-Synchronized Recordings

## Goals

* Learn how to validate VRS files recorded in a [TICSync](https://facebookresearch.github.io/projectaria_tools/docs/ARK/sdk/ticsync) session.
* Learn how fetch synchronized frames from TICSync VRS files.
* Learn how to interpret synchronization offsets amongst the frames of the recordings. 

## Understanding Time Domains

In a TICSync recording, all devices mark video frames with a timestamp in a conceptual TICSync time domain. During the recording, the TICSync algorithm constructs, on-the-fly, the mapping between the conceptual TICSync time domain and the concrete `DEVICE_TIME` time domains of the glasses. Under the current implementation, the unique _server_ device uses its `DEVICE_TIME` as the conceptual TICSync time, while all clients use their concrete `TIC_SYNC` time domains. The code below shows how to download TICSync sample recordings (VRS files) and how to query the concrete time domains in the VRS files.

## Python Dependencies

1. Set up a Python virtual environment with [this version of Projectaria Tools using pip](https://facebookresearch.github.io/projectaria_tools/docs/data_utilities/installation/installation_python)
2. You may have to `pip install matplotlib notebook==6.5.7`. Notebook v7 may have issues.
4. `cd ~ && jupyter notebook`.
5. Navigate in jupyter's file browser to the location of this notebook

Alternatively, try the notebook in colab.

## Download Sample TICSync VRS Files

We have three sample synchronized recordings of the same millisecond-resolution clock display. The recordings include 3 minutes of simultaneous recording. Some of the recordings are longer than three minutes long because the glasses take time to connect and they start recording sequentially.

The files are around 3&nbsp;GB each, so the downloading may take some considerable time. Check your `/tmp/ticsync_sample_data` folder to track download progress. The notebook kernel may appear frozen during the downloads, but it's not. The cell below will finish eventually. 

The logic checks whether the files have already been downloaded so you only have to wait once, then you can repeatedly run the notebook.

If you prefer, you may substitute your own TICSync files for our samples. Just bypass this downloading code (don't run the cells) and adjust the definition of `ticsync_pathnames` as needed.

In [ ]:
import os
from tqdm import tqdm
from urllib.request import urlretrieve
from zipfile import ZipFile

google_colab_env = 'google.colab' in str(get_ipython())
if google_colab_env:
    print("Running from Google Colab, installing projectaria_tools and getting sample data")
    !pip install projectaria-tools
    ticsync_sample_path = "./ticsync_sample_data/"
else:
    ticsync_sample_path = "/tmp/ticsync_sample_data/"

base_url = "https://www.projectaria.com/async/sample/download/?bucket=core&filename="
os.makedirs(ticsync_sample_path, exist_ok=True)

ticsync_filenames = [
    "ticsync_tutorial_server_3m.vrs",
    "ticsync_tutorial_client1_3m.vrs",
    "ticsync_tutorial_client2_3m.vrs",]

print("Downloading sample data (if they don't already exist)")
for filename in tqdm(ticsync_filenames):
    print(f"Processing: {filename}")
    full_path: str = os.path.join(ticsync_sample_path, filename)
    if os.path.isfile(full_path):
        print(f"{full_path} has alredy been downloaded.")
    else:
        print(f"Downloading {base_url}{filename} to {full_path}")
        urlretrieve(f"{base_url}{filename}", full_path)
        if filename.endswith(".zip"):
            with ZipFile(full_path, 'r') as zip_ref:
                zip_ref.extractall(path=ticsync_sample_path)                

## Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from projectaria_tools.core import data_provider
from projectaria_tools.core.sensor_data import (
    SensorData,
    TimeDomain,
    TimeQueryOptions,
)
from projectaria_tools.core.stream_id import StreamId

## Pathname Instructions

Adjust the following path names if necessary to accommodate the locations of the files you wish to analyze. These are the only names needed going forward.

In [ ]:
ticsync_pathnames = [
    os.path.join(ticsync_sample_path, filename)
    for filename in ticsync_filenames]

## Get Data Providers

In [ ]:
providers = [data_provider.create_vrs_data_provider(filename)
             for filename in ticsync_pathnames]

## Get and Browse Metadata

Let's examine the metadata for one of the providers.

The metadata are in a Python object. Here is a way to convert it (or any other object) into a dict for browsing its fields.

In [ ]:
def object_to_dict(object):
    import re
    dunder = re.compile(r"^__.*__$")  
    attributes = [member for member in dir(object)
                  if not dunder.match(member)]
    result = {attribute: getattr(object, attribute)
              for attribute in attributes}
    return result 

In [ ]:
server_metadata = providers[0].get_metadata()
object_to_dict(server_metadata)

Now that we know that the metadata have the attribute `time_sync_mode`, we can dot into it:

In [ ]:
server_metadata.time_sync_mode

Python `enums` like `time_sync_mode` have a `name` attribute that we can use for testing in code below. Here are all the attributes of the `time_sync_mode` attribute with all the possible values and the particular value pertaining to the server metadata.

In [ ]:
object_to_dict(server_metadata.time_sync_mode)

In [ ]:
server_metadata.time_sync_mode.name

## Check that all VRS Files Belong to the Same Session

One critical attribute of the metadata is the `shared_session_id`. Your shared recordings must belong to the same shared session. If they do not, the results are nonsense.

In [ ]:
def print_session_ids(providers):
    for provider in providers:
        print(f'shared session id = {provider.get_metadata().shared_session_id}')

def check_session_ids(providers):
    session_ids = [provider.get_metadata().shared_session_id
                  for provider in providers]
    assert (sid == session_ids[0] for sid in session_ids)

In [ ]:
print_session_ids(providers)
check_session_ids(providers)

We'll use `check_session_ids` in the display codes below.

## Displaying Frames by Timestamp(ns)

First define a `streams` dictionary, which reminds us that there are other synchronized recordings in the VRS files. We use only `"camera-rgb"` in this notebook.

After these definitions, we'll see how to investigate the synchronized VRS files by timestamp.

In [ ]:
# It's possible to search other image streams.
streams = {
    "camera-slam-left": StreamId("1201-1"),
    "camera-slam-right":StreamId("1201-2"),
    "camera-rgb":StreamId("214-1"),
    "camera-eyetracking":StreamId("211-1"),}

In [ ]:
def get_server_provider(providers):
    server_providers = [provider for provider in providers
                       if provider.get_metadata().time_sync_mode.name == 'TicSyncServer']
    return server_providers[0]

In [ ]:
server_provider = get_server_provider(providers)

In [ ]:
all_server_timestamps_ns = server_provider.get_timestamps_ns(
    streams["camera-rgb"], TimeDomain.DEVICE_TIME)

### Helper Functions

In [ ]:
MS_PER_NS = 1 / 1_000_000

def ticsync_time_domain_from_provider(provider):
    """Return a VRS file's local approximation of the conceptual 
    TICSync time."""
    mode = provider.get_metadata().time_sync_mode.name
    if mode == 'TicSyncServer':
        domain = TimeDomain.DEVICE_TIME
    elif mode == 'TicSyncClient':
        domain = TimeDomain.TIC_SYNC
    else:
        raise NotImplementedError(f'Unsupported time-sync mode {mode}')
    return domain

def split_providers(providers):
    """A utility function used internally."""
    server_provider = [provider for provider in providers
                       if provider.get_metadata().time_sync_mode.name 
                          == 'TicSyncServer'][0]
    client_providers = [provider for provider in providers
                       if provider.get_metadata().time_sync_mode.name 
                          == 'TicSyncClient']
    return server_provider, client_providers
    
def print_timestamp_offsets_ms(time_ns, providers):
    """We are concerned with the offsets (time differences) between
    client glasses and server glasses. Offsets between clients are
    not informative, as each client settles to an approximation of
    the server's timestamps."""
    server_provider, client_providers = split_providers(providers)
    server_time_ns = get_closest_timestamp_ns(time_ns, server_provider)
    client_times_ns = [get_closest_timestamp_ns(time_ns, client_provider)
                       for client_provider in client_providers]
    for i, client_time_ns in enumerate(client_times_ns):
        offset = (client_time_ns - server_time_ns) * MS_PER_NS
        print(f'client{i + 1} offset (ms) = {offset}')

def get_closest_timestamp_ns(ticsync_time_ns, provider):
    """Return the actual timestamp in a VRS file that's closest
    to a given time in nanoseconds."""
    domain = ticsync_time_domain_from_provider(provider)
    return provider.get_sensor_data_by_time_ns(
        stream_id=streams["camera-rgb"],
        time_ns=ticsync_time_ns,
        time_domain=domain,
        time_query_options=TimeQueryOptions.CLOSEST).get_time_ns(domain)
    
def get_closest_image_by_ticsync_time(ticsync_time_ns, provider):
    """Get an image from a VRS file closest in TICSync time to a 
    given time in nanoseconds."""
    return provider.get_image_data_by_time_ns(
        stream_id=streams["camera-rgb"],
        time_ns=ticsync_time_ns,
        time_domain=ticsync_time_domain_from_provider(provider),
        time_query_options=TimeQueryOptions.CLOSEST)

### Show Frames by Timestamp

In [ ]:
def show_frames_by_ticsync_timestamp_ns(ticsync_time_ns, providers):
    check_session_ids(providers)
    images = [get_closest_image_by_ticsync_time(ticsync_time_ns, provider) 
             for provider in providers]
    print_timestamp_offsets_ms(ticsync_time_ns, providers)
    fig_m, axes_m = plt.subplots(1, len(providers), figsize=(10, 5), dpi=300)
    image_index = 0
    for idx, frame in enumerate(images):
        axes_m[idx].set_title(providers[idx].get_metadata().time_sync_mode.name)
        npa = frame[0].to_numpy_array()
        npar = np.rot90(npa, k=1, axes=(1, 0))
        axes_m[idx].imshow(npar)
    plt.show()

### At an Arbitrary Timestamp

In [ ]:
show_frames_by_ticsync_timestamp_ns(all_server_timestamps_ns[len(all_server_timestamps_ns) // 2], providers)

See that they're synchronized to clock time within 16 ms, within one frame of each other.

## Waiting for TICSync Settling

TICSync needs warmup, typically 45 seconds after recording starts for each device to settle. Here is code to show you how to find timestamps before and after this settling time.

In [ ]:
SEC_PER_NS = 1 / 1e9

def diff_timestamps_ns_s(t1_ns, t2_ns):
    return (t1_ns - t2_ns) * SEC_PER_NS

def timestamp_ns_after_delay_s(timestamps_ns, delay_s):
    first_timestamp_ns = timestamps_ns[0]
    for i, ts_ns in enumerate(timestamps_ns):
        if diff_timestamps_ns_s(ts_ns, first_timestamp_ns) >= delay_s:
            break
    return ts_ns

The TICSync time after 45 seconds since last device began recording.

In [ ]:
ticsync_time_ns_after_settlement = max([timestamp_ns_after_delay_s(
    provider.get_timestamps_ns(
        streams["camera-rgb"], 
        ticsync_time_domain_from_provider(provider)), 45) 
    for provider in providers])

That allows us to display the first frames after the delay.

In [ ]:
show_frames_by_ticsync_timestamp_ns(ticsync_time_ns_after_settlement, providers)

Observe, again, that the offsets are within one frame.

## Conclusion

Generally speaking, TICSync performs within 1 frame after a 45-second settling time.

We have exhibited general tools for displaying and manipulating synchronized data from VRS files. We have shown how to assess the synchronization versus a physical time standard such as a millisecond clock display. 